In [2]:
import numpy as np
import pandas as pd
import os
import pickle

In [3]:
folder_path = "../../../../../../../Volumes/T7 Shield/exp_3"
# experiment 1
file_prefix = "ORCA025.L46.LIM2vp.CFCSF6.MOPS.JRA.LP04-KLP002.hind_"
# experiment 3: "ORCA025.L46.LIM2vp.CFCSF6.MOPS.JRA.LP11-KLP009.clim_"
# experiment 5: "ORCA025.L46.LIM2vp.CFCSF6.MOPS.JRA.LP04-KLP002.wind_"
# experiment 6: "ORCA025.L46.LIM2vp.CFCSF6.MOPS.JRA.L.LP03-KLP004_"


file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".pkl")])
# file_list = file_list[:2]

# Columns to compute statistics for (excluding weights and masks)
columns_to_process = [
    'SST', 'SAL', 'ice_frac', 'mixed_layer_depth', 'heat_flux_down', 'water_flux_up',
    'stress_X', 'stress_Y', 'currents_X', 'currents_Y', 'co2flux', 'co2flux_pre'
]

# Initialize statistics dictionary
stats = {
    col: {
        "weighted_sum": 0.0,
        "weighted_sum_sq": 0.0,
        "total_weight": 0.0,
        "min": float("inf"),
        "max": float("-inf")
    } for col in columns_to_process
}

# Loop through files
for filename in file_list:
    filepath = os.path.join(folder_path, filename)

    with open(filepath, "rb") as f:
        df = pickle.load(f)

    # Filter valid cells
    mask = df["tmask"] == 1
    
    # Compute weights = area
    weights = (df["e1t"] * df["e2t"])[mask]

    for col in stats:
        values = df[col][mask].values
        w = weights.values

        weighted_sum = np.sum(values * w)
        weighted_sum_sq = np.sum((values ** 2) * w)
        total_weight = np.sum(w)

        stats[col]["weighted_sum"] += weighted_sum
        stats[col]["weighted_sum_sq"] += weighted_sum_sq
        stats[col]["total_weight"] += total_weight
        stats[col]["min"] = min(stats[col]["min"], values.min())
        stats[col]["max"] = max(stats[col]["max"], values.max())

# Compute final weighted statistics
results = {}
for col, s in stats.items():
    if s["total_weight"] == 0:
        continue
    weighted_mean = s["weighted_sum"] / s["total_weight"]
    weighted_var = (s["weighted_sum_sq"] / s["total_weight"]) - weighted_mean ** 2
    weighted_std = np.sqrt(weighted_var)
    results[col] = {
        "min": s["min"],
        "max": s["max"],
        "mean (area-weighted)": weighted_mean,
        "std_dev (area-weighted)": weighted_std
    }

# Print results
for col, res in results.items():
    print(f"\nStatistics for {col}:")
    for k, v in res.items():
         print(f"  {k}: {round(v, 4)}")


Statistics for SST:
  min: -2.0220000743865967
  max: 34.574798583984375
  mean (area-weighted): 18.4445
  std_dev (area-weighted): 10.2876

Statistics for SAL:
  min: 0.0003000000142492354
  max: 42.55849838256836
  mean (area-weighted): 34.6651
  std_dev (area-weighted): 1.8331

Statistics for ice_frac:
  min: 0.0
  max: 1.0
  mean (area-weighted): 0.054
  std_dev (area-weighted): 0.2101

Statistics for mixed_layer_depth:
  min: 12.839099884033203
  max: 3488.6279296875
  mean (area-weighted): 49.2615
  std_dev (area-weighted): 61.9695

Statistics for heat_flux_down:
  min: -1316.387939453125
  max: 296.58331298828125
  mean (area-weighted): -0.1551
  std_dev (area-weighted): 89.8908

Statistics for water_flux_up:
  min: -0.050200000405311584
  max: 0.024399999529123306
  mean (area-weighted): 0.0
  std_dev (area-weighted): 0.0002

Statistics for stress_X:
  min: -0.6261000037193298
  max: 0.5073000192642212
  mean (area-weighted): 0.0072
  std_dev (area-weighted): 0.074

Statistics